## Section 1: Similarity Computations

### 1.a Jaccard Similarity (Users)

In [21]:
import numpy as np
#from sklearn.metrics import jaccard_score
from itertools import combinations

In [12]:
users_watch={   "Willy":  [1,1,1,0,1],
                "Xavier": [1,1,1,0,0],
                "Yvonne": [0,1,1,1,0],
                "Zac":    [1,1,0,1,1]
            }
def jaccardscore(u1,u2,matrix):
    user1=matrix[u1]
    user2=matrix[u2]
    intersection = sum([i & j for i, j in zip(user1, user2)])
    union = sum([i | j for i ,j  in zip(user1,user2)])
    return intersection/union if union !=0 else 0
# 所有使用者名稱
user_names = list(users_watch.keys())
user_watch_jaccardscore={}
# 計算兩兩組合的 Jaccard 相似度
for u1, u2 in combinations(user_names, 2):
    score = jaccardscore(u1, u2,users_watch)
    pair=f"{u1} & {u2}:"
    user_watch_jaccardscore[pair] = score
    #print(f"{u1} & {u2} : Jaccard similarity = {score:.2f}")
max_score=max(user_watch_jaccardscore.values())
max_score_pair=[pair for pair,score in user_watch_jaccardscore.items() if score==max_score]
print("最相似的使用者:\n")
for i in max_score_pair:
    print(f"{i}:{max_score}")    

最相似的使用者:

Willy & Xavier::0.75


### 1.b Jaccard Similarity (Movies)

In [13]:
movie_watch={   "La La Land":    [1,1,0,1],
                "The Lion King": [1,1,1,1],
                "Parasite":      [1,1,1,0],
                "Joker":         [0,0,1,1],
                "John Wick3":    [1,0,0,1]
            }
movie_names = list(movie_watch.keys())
movie_watch_jaccardscore={}

for u1, u2 in combinations(movie_names, 2):
    score = jaccardscore(u1, u2,movie_watch)
    pair=f"{u1} & {u2}:"
    movie_watch_jaccardscore[pair] = score

max_score=max(movie_watch_jaccardscore.values())
max_score_pair=[pair for pair,score in movie_watch_jaccardscore.items() if score==max_score]
print("最相似的電影:\n")
for i in max_score_pair:
    print(f"{i}:{max_score}")    

最相似的電影:

La La Land & The Lion King::0.75
The Lion King & Parasite::0.75


### 1.c Pearson Correlation (Users)

In [14]:
from scipy.stats import pearsonr

In [16]:
user_rating={
    "Willy"  : [5,5,4,None,4],
    "Xavier" : [1,2,2,None,None],
    "Yvonne" : [None,2,1,5,None],
    "Zac"    : [2,2,None,1,1]
}
def pearson_similarity(u1, u2):
    r1 = user_rating[u1]
    r2 = user_rating[u2]
    
    # 只保留兩人都評分的電影
    paired_ratings = [(x, y) for x, y in zip(r1, r2) if x is not None and y is not None]
    if len(paired_ratings) < 2:
        return 0  # 太少資料無法算 Pearson
    r1,r2 = zip(*paired_ratings)
    
    return pearsonr(r1, r2)[0]  # Pearson correlation

user_names = list(user_rating.keys())
pearson_scores = {}

for u1, u2 in combinations(user_names, 2):
    score = pearson_similarity(u1, u2)
    pearson_scores[f"{u1} & {u2}"] = score

# 找出最大值
max_score = max(pearson_scores.values())
most_similar_pairs = [pair for pair, score in pearson_scores.items() if score == max_score]

print("\nPearson 最相似的使用者配對：")
for pair in most_similar_pairs:
    print(f"{pair} : {max_score:.2f}")


Pearson 最相似的使用者配對：
Willy & Yvonne : 1.00
Willy & Zac : 1.00


C:\Users\user\AppData\Local\Temp\ipykernel_2408\2038560187.py:17: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return pearsonr(r1, r2)[0]  # Pearson correlation


## Section 2: Collaborative Filtering

### 2.a

#### A:T(ii)的意義為使用者i喜歡T(ii)個項目，T(ij)的意義為使用者i和使用者j同時喜歡T(ij)個項目。

### 2.b

### A:見pdf

### 2.c

### A:見pdf

### 2.d Application to Real Data

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import heapq

In [26]:
user_show=[]
with open('user-shows.txt') as users:
    for line in users:
        user_show.append([int(x) for x in line.split()])
shows=[]
with open('shows.txt') as show:
    for line in show:
        shows.append(line.strip())

# Jim前100部節目設為0
user_show_masked = user_show.copy()
jim = user_show[499].copy()
for i in range(100):
    jim[i] = 0
user_show_masked[499] = jim
# 轉NumPy
R=np.array(user_show_masked)
# 計算 cosine similarity
S_u=cosine_similarity(R)
# 預測分數矩陣 Γ_U
Gamma_U = S_u@R
# 取出 Jim 的推薦分數（前100個）
jim_predict = Gamma_U[499]
# 推薦前 5 部節目
top_5=heapq.nlargest(5, range(100), key=lambda i: jim_predict[i])
for i in top_5:
    print(f"{shows[i]}:預測分數 {jim_predict[i]:.2f}")


"FOX 28 News at 10pm":預測分數 908.48
"Family Guy":預測分數 861.18
"2009 NCAA Basketball Tournament":預測分數 827.60
"NBC 4 at Eleven":預測分數 784.78
"Two and a Half Men":預測分數 757.60


In [29]:
# 使用者-項目矩陣
R = np.array(user_show)
# 先備份 Jim 原始行為（之後要比對正確性）
jim_original = R[499].copy()

# 將前 100 部節目設為未知（masking）
R[499, :100] = 0

# 每個「項目」是 R 的一欄
# 轉置後，每列代表一個項目，計算 item-item cosine
S_i = cosine_similarity(R.T)

# 計算 Jim 的推薦分數（向量）
jim_scores_item = R[499] @ S_i  # shape: (num_items,)

# 只挑前 100 部節目
top_100_scores = jim_scores_item[:100]

# 取推薦分數前 5 高的節目 index
top_5= heapq.nlargest(5, range(100), key=lambda i: top_100_scores[i])

# 顯示推薦結果
print("項目－項目協同過濾推薦：")
for idx in top_5:
    print(f"{shows[idx]}:預測分數 {top_100_scores[idx]:.2f}")


項目－項目協同過濾推薦：
"FOX 28 News at 10pm":預測分數 31.36
"Family Guy":預測分數 30.00
"NBC 4 at Eleven":預測分數 29.40
"2009 NCAA Basketball Tournament":預測分數 29.23
"Access Hollywood":預測分數 28.97


## Section 3: Latent Factor Recommendation

### 3.b Implementation and Evaluation